In [45]:
import pandas as pd
import json
import numpy as np
import os
from google.cloud import storage
from io import BytesIO
from pypdf import PdfReader  # You may need to install pypdf if you haven't already
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
#from langchain.llms import OpenAI
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from langchain_google_vertexai import ChatVertexAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import logging

In [46]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'xenon-heading-430209-e4-7582f64a7330.json' # Rellenar con el .json correspondiente
logging.basicConfig(filename='error_log.txt', level=logging.ERROR, format='%(asctime)s %(message)s')

In [47]:
def leer_blob_en_memoria(bucket_name, blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    pdf_content = blob.download_as_bytes()
    return pdf_content

def extraer_texto_de_pdf_bytes(pdf_bytes):
    pdf_file = BytesIO(pdf_bytes)
    reader = PdfReader(pdf_file)
    text = ''
    for page in reader.pages:
        text += page.extract_text() + '\n'
    return text

def procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name):
    pdf_bytes = leer_blob_en_memoria(bucket_name, blob_name)
    content = extraer_texto_de_pdf_bytes(pdf_bytes)
    return content

def listar_pdfs(bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blobs = bucket.list_blobs()

    # Filtra los archivos PDF y guarda sus URLs completas en la lista
    listado_pdfs = [f"gs://{bucket_name}/{blob.name}" for blob in blobs if blob.name.endswith('.pdf') and not blob.name.startswith('resoluciones/')]
    return listado_pdfs

def dividir_lista_pdfs(listado_pdfs):
    total = len(listado_pdfs)
    tamaño_parte = total // 5
    
    parte1 = listado_pdfs[:tamaño_parte]
    parte2 = listado_pdfs[tamaño_parte:2*tamaño_parte]
    parte3 = listado_pdfs[2*tamaño_parte:3*tamaño_parte]
    parte4 = listado_pdfs[3*tamaño_parte:4*tamaño_parte]
    parte5 = listado_pdfs[4*tamaño_parte:]
    
    return parte1, parte2, parte3, parte4, parte5

In [48]:
bucket_name = 'tfm_javi'

In [51]:
# Definir los esquemas de respuesta
response_schemas = [
    ResponseSchema(name="numero_expediente", description="Número de Expediente, ejemplo SD2023/0000046"),
    ResponseSchema(name="resolucion", description="Una de las siguientes opciones: negada_con_oposicion, negada_sin_oposicion, aprobada_sin_oposicion, aprobada_con_oposición"),
    ResponseSchema(name="numero_de_resolución", description="Número entero de la resolución, ejemplo 2195"),
    ResponseSchema(name="denominacion", description="Nombre de la empresa que solicita el registro de la marca"),
    ResponseSchema(name="vigencia", description="Fecha en que expira la vigencia del registro, o texto si es negada o vencida"),
    ResponseSchema(name="titular", description="Titular de la marca que intenta registrar"),
    ResponseSchema(name="clase", description="Número o lista de números de la Clasificación Internacional de Niza, por ejemplo [42, 35, 27]"),
    ResponseSchema(name="gaceta", description="Número de la gaceta de Propiedad Industrial donde se publica"),
    ResponseSchema(name="tipo", description="Tipo de registro que se intenta realizar, por ejemplo Mixta, Nominativa, Figurativa"),
    ResponseSchema(name="fecha_solicitud", description="Fecha de presentación de la solicitud"),
    ResponseSchema(name="fecha_resolucion", description="Fecha de resolución"),
    ResponseSchema(name="nombre_opositor", description="Nombre de la empresa que se opone a la publicación"),
    ResponseSchema(name="signo_opositor_opositores", description="Signo o signos de los opositores en conflicto"),
    ResponseSchema(name="argumento_oposición", description="Argumentos en los que se basa para oponerse al registro y artículos en los que se apoya"),
    ResponseSchema(name="explicacion_argumentos_oposicion", description="Breve resumen y explicación de los argumentos de la oposición"),
    ResponseSchema(name="resolucion_organismo", description="Resolución del organismo competente, por ejemplo: 'DENIEGA el registro de la marca PAPELES LA FAVORITA (Mixta)'")
]


In [52]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
# Escapar las llaves para evitar la interpretación de variables
format_instructions = format_instructions.replace("{", "{{").replace("}", "}}")

In [53]:
system_prompt = """
Eres un Experto abogado Colombiano en analizar resoluciones del SIC (Superintendencia de Industria y Comercio de Colombia),en el ámbito de registro de marcas y lemas.
quiero que extraigas el numero de Expediente, la resolución del conflicto, el numero de la resolución, el nombre de la marca que intenta registrarse, el titular que intenta registrar la marca, el numero de clase que intenta registrar, 
el numero de la gaceta en que ha sido publicada, la fecha de solicitud de registro, nombre de la empresa opositora, el titular de la empresa que se opone si aparece, y los argumentos de derecho en los que se apoya el opositor.
"""
human_prompt = f"""Extrae la información indicada en DATOS a partir del TEXTO de la resolución

TEXT
---
\n\n{{contenido_pdf}}
---

DATOS
{format_instructions}
"""


In [54]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template(human_prompt),
])

In [55]:
# Define tu LLM 
llm = ChatVertexAI(
    model="gemini-1.5-pro-001"
      #verificar la relacion megas tokens  
)

In [56]:
chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)


In [57]:
# Obtener la lista de PDFs
listado_pdfs = listar_pdfs(bucket_name)

In [ ]:
len(listado_pdfs)

In [60]:
parte1, parte2, parte3, parte4, parte5 = dividir_lista_pdfs(listado_pdfs)

In [ ]:
parte4

In [19]:
# Inicializar una lista para almacenar los resultados
resultados = []


In [ ]:
 for pdf_uri in parte4:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

## Errores de analisis
realizamos en la terminal cat error_log.txt que nos da el resultado de:

error al procesar resoluciones/NCO-SD2022-0073142.pdf

error al procesar resoluciones/NCO-SD2022-0113402.pdf

error al procesar resoluciones/SD2022-0017210.pdf

vamos  a realizar un intento individual con esos documentos

In [ ]:
NCO-SD2018-0100970.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 16 column 134 (char 1234)
2024-10-02 08:49:01,595 Ocurri� un error al procesar NCO-SD2019-0004850.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 15 column 695 (char 1120)
2024-10-02 08:49:12,863 Ocurri� un error al procesar NCO-SD2019-0004867.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 16 column 184 (char 1328)
2024-10-02 10:13:46,462 Ocurri� un error al procesar NCO-SD2022-0068161.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 16 column 202 (char 1352)
2024-10-02 10:25:58,614 Ocurri� un error al procesar NCO-SD2022-0124302.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 15 column 318 (char 788)
2024-10-02 10:39:28,243 Ocurri� un error al procesar NCO-SD2023-0041476.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 15 column 549 (char 1023)
2024-10-02 10:46:26,646 El contenido de NSO-13290945.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:27,672 El contenido de NSO-13301940.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:28,676 El contenido de NSO-14001188.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:29,557 El contenido de NSO-14001216.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:30,622 El contenido de NSO-14001244.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:31,632 El contenido de NSO-14003030.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:32,592 El contenido de NSO-14003064.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:33,441 El contenido de NSO-14019282.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:34,089 El contenido de NSO-14043699.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:34,813 El contenido de NSO-14049638.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:35,818 El contenido de NSO-14056511.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:36,552 El contenido de NSO-14081279.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:37,405 El contenido de NSO-14083632.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:38,508 El contenido de NSO-14083668.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:39,502 El contenido de NSO-14083716.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:40,606 El contenido de NSO-14083737.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:46:41,608 El contenido de NSO-14218670.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:47:09,626 El contenido de NSO-15275357.pdf est� vac�o o no se pudo extraer texto.
2024-10-02 10:53:53,246 Ocurri� un error al procesar NSO-16054865.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 16 column 130 (char 1647)
2024-10-02 11:07:22,035 Ocurri� un error al procesar NSO-16175054.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 17 column 1 (char 1663)
2024-10-02 11:07:56,452 Ocurri� un error al procesar NSO-SD2016-0000475.pdf: Got invalid return object. Expected key `resolucion_organismo` to be present, but got {'numero_expediente': 'SD2016/0000475', 'resolucion': 'negada_sin_oposicion', 'numero_de_resoluci�n': '83213', 'denominacion': 'UNIT', 'vigencia': None, 'titular': 'BIOTECHNOLOGY INSTITUTE, S.L.', 'clase': '10', 'gaceta': '774', 'tipo': 'Nominativa', 'fecha_solicitud': '8 de marzo de 2016', 'fecha_resolucion': '13 de diciembre de 2017', 'nombre_opositor': None, 'signo_opositor_opositores': 'UNIKKAVO', 'argumento_oposici�n': 'El Director de Signos Distintivos, en virtud del art�culo 150 de la Decisi�n 486 de la Comunidad Andina, realiz� un estudio de registrabilidad de oficio, a pesar de no haberse presentado oposiciones de terceros. La causal de irregistrabilidad aplicada fue el Literal a) del art�culo 136 de la Decisi�n 486, la cual proh�be el registro de marcas que puedan generar un riesgo de confusi�n o asociaci�n con marcas anteriores. En este caso, se argument� que la marca solicitada UNIT es similarmente confundible con la marca registrada UNIKKAVO, ambas para productos de la clase 10 (aparatos e instrumentos odontol�gicos). Se consider� que la similitud ortogr�fica y fon�tica entre UNIT y UNIK, sin elementos distintivos adicionales, podr�a inducir a error a los consumidores, quienes podr�an percibir la marca solicitada como una extensi�n o variaci�n de la marca registrada.', 'explicacion_argumentos_oposicion': 'La marca solicitada fue negada debido a su similitud con la marca registrada UNIKKAVO, lo que podr�a causar confusi�n en el mercado. La decisi�n se bas� en la legislaci�n andina sobre propiedad industrial, espec�ficamente en las disposiciones que proh�ben el registro de marcas que puedan generar confusi�n con marcas preexistentes.'}
2024-10-02 11:47:56,088 Ocurri� un error al procesar NSO-SD2016-0056492.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 16 column 265 (char 1530)
2024-10-02 11:54:20,613 Ocurri� un error al procesar NSO-SD2017-0001216.pdf: Got invalid return object. Expected key `resolucion_organismo` to be present, but got {'numero_expediente': 'SD2017/0001216', 'resolucion': 'negada_sin_oposicion', 'numero_de_resoluci�n': '40996', 'denominacion': 'NEXCEL', 'vigencia': None, 'titular': 'Castrol Limited', 'clase': '[4, 7, 37, 40]', 'gaceta': '787', 'tipo': 'Nominativa', 'fecha_solicitud': '4 de diciembre de 2015', 'fecha_resolucion': '11 de julio de 2017', 'nombre_opositor': None, 'signo_opositor_opositores': 'EXCEL-GKYB CORPORATION, EXCELETERNIT COLOMBIANA S.A., NEXEN NEXEN INC.', 'argumento_oposici�n': 'La Superintendencia de Industria y Comercio neg� el registro de la marca NEXCEL de manera oficiosa bas�ndose en el Art�culo 136 literal a) de la Decisi�n 486 de la Comisi�n de la Comunidad Andina. Argument� que exist�a un riesgo de confusi�n o asociaci�n con las marcas previamente registradas EXCEL-GKYB CORPORATION, EXCELETERNIT COLOMBIANA S.A., NEXEN NEXEN INC. debido a la similitud ortogr�fica y fon�tica entre los signos. Adem�s, se consider� que exist�a una conexi�n competitiva entre los productos y servicios que distinguen los signos en conflicto, ya que se complementan entre s� y se dirigen al mismo tipo de consumidor.', 'explicacion_argumentos_oposicion': 'A pesar de no haber oposici�n de terceros, la SIC neg� el registro de la marca NEXCEL por similitud con marcas registradas que podr�an generar confusi�n en el consumidor. Se consider� que la marca solicitada era ortogr�fica y fon�ticamente similar a las marcas preexistentes, y que exist�a una conexi�n competitiva entre los productos y servicios cubiertos por las marcas, lo que incrementaba el riesgo de asociaci�n indebida.'}
2024-10-02 11:57:04,937 Ocurri� un error al procesar NSO-SD2017-0006988.pdf: Got invalid return object. Expected key `resolucion_organismo` to be present, but got {'numero_expediente': 'SD2017/0006988', 'resolucion': 'negada_sin_oposicion', 'numero_de_resoluci�n': '46401', 'denominacion': 'nektium', 'vigencia': 'No aplica', 'titular': 'POLIFENOLES NATURALES, S.L.', 'clase': '5', 'gaceta': '792', 'tipo': 'Mixta', 'fecha_solicitud': '24 de junio de 2016', 'fecha_resolucion': '31 de julio de 2017', 'nombre_opositor': None, 'signo_opositor_opositores': 'KENTIUM DIGEST\nKENTIUM', 'argumento_oposici�n': 'La Direcci�n de Signos Distintivos encontr� que la marca solicitada "nektium" es similarmente confundible con las marcas previamente registradas "KENTIUM" y "KENTIUM DIGEST", argumentando que:\n\n* **Similitud ortogr�fica y fon�tica:** Ambos signos comparten 7 letras y 3 s�labas, coincidiendo en la s�laba t�nica "TI-UM". El cambio de posici�n entre las letras "N" y "K" no es suficiente para generar distintividad.\n* **Elemento nominativo predominante:** Aunque la marca solicitada es mixta, el elemento nominativo predomina sobre el gr�fico, generando mayor recordaci�n en el consumidor. \n* **Conexi�n competitiva:** Se identific� que los productos que se pretenden distinguir con la marca solicitada (clase 5) son similares a los productos identificados por las marcas previamente registradas (tambi�n clase 5), lo que genera un riesgo de confusi�n en el consumidor.\n\nEn base a estos argumentos, se determin� que la marca solicitada "nektium" incurr�a en la causal de irregistrabilidad establecida en el **art�culo 136 literal a) de la Decisi�n 486 de la Comisi�n de la Comunidad Andina**, la cual proh�be el registro de marcas que puedan generar un riesgo de confusi�n con marcas previamente registradas para productos o servicios similares.', 'explicacion_argumentos_oposicion': 'La Superintendencia neg� el registro de la marca "nektium" por similitud con las marcas registradas "KENTIUM" y "KENTIUM DIGEST", argumentando similitud ortogr�fica, fon�tica y conexidad competitiva entre los productos de clase 5 que ambas marcas identifican. Se consider� que exist�a un alto riesgo de confusi�n para el consumidor, infringiendo el art�culo 136 literal a) de la Decisi�n 486.'}
2024-10-02 12:23:20,040 Ocurri� un error al procesar NSO-SD2017-0030838.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 16 column 76 (char 1216)
2024-10-02 12:44:30,500 Ocurri� un error al procesar NSO-SD2017-0044521.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 15 column 344 (char 814)
2024-10-02 13:13:41,652 Ocurri� un error al procesar NSO-SD2017-0069131.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 16 column 76 (char 1166)
2024-10-02 13:26:28,139 Ocurri� un error al procesar NSO-SD2017-0079074.pdf: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)
2024-10-02 13:29:41,193 Ocurri� un error al procesar NSO-SD2017-0081349.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 16 column 76 (char 984)
2024-10-02 13:42:32,195 Ocurri� un error al procesar NSO-SD2017-0089366.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 16 column 192 (char 1191)
2024-10-02 14:03:45,945 Ocurri� un error al procesar NSO-SD2018-0001113.pdf: Got invalid JSON object. Error: Expecting ',' delimiter: line 16 column 195 (char 1273)
2024-10-02 15:02:48,464 Ocurri� un error al procesar NSO-SD2018-0054263.pdf:

In [71]:
listado_pdfs_error = (
    # f"gs://{bucket_name}/NCO-SD2018-0100970.pdf",
    # f"gs://{bucket_name}/NCO-SD2019-0004850.pdf",
    # f"gs://{bucket_name}/NCO-SD2019-0004867.pdf",
    # f"gs://{bucket_name}/NCO-SD2022-0068161.pdf",
    # f"gs://{bucket_name}/NCO-SD2022-0124302.pdf",
    #f"gs://{bucket_name}/NCO-SD2023-0041476.pdf",
    # f"gs://{bucket_name}/NSO-13290945.pdf",
    # f"gs://{bucket_name}/NSO-13301940.pdf",
    # f"gs://{bucket_name}/NSO-14001188.pdf",
    # f"gs://{bucket_name}/NSO-14001216.pdf",
    # f"gs://{bucket_name}/NSO-14001244.pdf",
    # f"gs://{bucket_name}/NSO-14003030.pdf",
    # f"gs://{bucket_name}/NSO-14003064.pdf",
    # f"gs://{bucket_name}/NSO-14019282.pdf",
    # f"gs://{bucket_name}/NSO-14043699.pdf",
    # f"gs://{bucket_name}/NSO-14049638.pdf",
    # f"gs://{bucket_name}/NSO-14056511.pdf",
    # f"gs://{bucket_name}/NSO-14081279.pdf",
    # f"gs://{bucket_name}/NSO-14083632.pdf",
    # f"gs://{bucket_name}/NSO-14083668.pdf",
    # f"gs://{bucket_name}/NSO-14083716.pdf",
    # f"gs://{bucket_name}/NSO-14083737.pdf",
    # f"gs://{bucket_name}/NSO-14218670.pdf",
    # f"gs://{bucket_name}/NSO-15275357.pdf",
    # f"gs://{bucket_name}/NSO-16054865.pdf",
    # f"gs://{bucket_name}/NSO-16175054.pdf",
    # f"gs://{bucket_name}/NSO-SD2016-0000475.pdf",
    #f"gs://{bucket_name}/NCO-SD2016-0056492.pdf",
    #f"gs://{bucket_name}/NSO-SD2017-0001216.pdf",
    #f"gs://{bucket_name}/NCO-SD2017-0006988.pdf",
    #f"gs://{bucket_name}/NCO-SD2017-0030838.pdf",
    #f"gs://{bucket_name}/NCO-SD2017-0044521.pdf",
    f"gs://{bucket_name}/NCO-SD2017-0069131.pdf",
    f"gs://{bucket_name}/NCO-SD2017-0079074.pdf",
    f"gs://{bucket_name}/NCO-SD2017-0081349.pdf",
    f"gs://{bucket_name}/NCO-SD2017-0089366.pdf",
    f"gs://{bucket_name}/NCO-SD2018-0001113.pdf",
    f"gs://{bucket_name}/NCO-SD2018-0054263.pdf",
)

In [ ]:
len(listado_pdfs_error)

In [ ]:
listado_pdfs_error

In [ ]:
 for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

Quiero volver a intentar este registro CSO-SD2022-0105220.pdf

Parece que esta vez si que ha procesado bien los tres archivos que diéron error, vamos a construir un df con los resultados

In [75]:
df_resultados = pd.DataFrame(resultados)

In [ ]:
df_resultados

In [77]:
pd.set_option('display.max_columns', None)

In [78]:
df_ai_annotations_full_3_4 = pd.DataFrame.from_records(df_resultados['text'])

In [ ]:
df_ai_annotations_full_3_4

In [ ]:
df_ai_annotations_full_1.info()

In [ ]:
df_ai_annotations["argumento_oposición"]

In [80]:
df_ai_annotations_full_3_4.to_csv("datasetia_full_3_4.csv")

In [6]:
df = pd.read_csv("datasetia.csv")

In [7]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
df

***JDS:*** *veo haciendo una exploración que XING FA es una marca que al parecer ya esta registrada, y que la SIC ha entrado de oficio en un caso (expediente SD2022/0011834) porque entra en conflicto con esa marca, hago una exploración muy basica a ver si es uno de los casos que ya hemos procesado y aparentemete no, es posible que si que este pero habria que normalizarlo a minusculas etc, pero tambien es muy posible que no este al ser solo 419 elementos del total, habria que hacer un merge con la bbdd del sql que nos dieron de sysmarck*

In [ ]:
df[df["denominacion"].str.contains("XING FA", case=False, na=False)]